## 01. Получаем данные из Harry Potter Community Wiki

В качестве источника данных будем использовать [Harry Potter Fandom Wiki](https://harrypotter.fandom.com/). Для удобства скачивания информации найдем ссылку на перечень всех страниц.

In [15]:
import requests

url = "https://harrypotter.fandom.com/ru/wiki/%D0%A1%D0%BB%D1%83%D0%B6%D0%B5%D0%B1%D0%BD%D0%B0%D1%8F:%D0%92%D1%81%D0%B5_%D1%81%D1%82%D1%80%D0%B0%D0%BD%D0%B8%D1%86%D1%8B"
res = requests.get(url)
index_page = res.text

Внутри странички все ссылки находятся внутри тега `mw-allpages-body`.

In [34]:
import re

def within(text,op,clo):
    return re.findall(f'{op}(.*?){clo}',text)

def get_div_class(html,cls):
    return re.findall(f'<div class="{cls}">(.*?)</div>',html)

def get_attr(attr,text,idx=0):
    return re.findall(f'{attr}="(.*?)"',text)[idx]

#get_div_class(txt,"mw-allpages-nav")
get_div_class(txt,"mw-allpages-body")

[]

In [38]:
link = "https://harrypotter.fandom.com/ru/wiki/%D0%A1%D0%BB%D1%83%D0%B6%D0%B5%D0%B1%D0%BD%D0%B0%D1%8F:%D0%92%D1%81%D0%B5_%D1%81%D1%82%D1%80%D0%B0%D0%BD%D0%B8%D1%86%D1%8B"

pages = {}
visited = set()

while link:
    if link in visited:
        print(f'Already visited: {link}')
        break
    visited.add(link)
    print(f'Getting {link}..',end='')
    html = requests.get(link).text
    linx = re.findall(f'<li>(.*?)</li>',html)
    print(f"{len(linx)} meaningful links")
    for x in linx:
        pages[get_attr("href",x)] = get_attr("title",x)
    link = "https://harrypotter.fandom.com" + get_attr("href",get_div_class(html,"mw-allpages-nav")[0],idx=-1)

Getting https://harrypotter.fandom.com/ru/wiki/%D0%A1%D0%BB%D1%83%D0%B6%D0%B5%D0%B1%D0%BD%D0%B0%D1%8F:%D0%92%D1%81%D0%B5_%D1%81%D1%82%D1%80%D0%B0%D0%BD%D0%B8%D1%86%D1%8B..48 meaningful links
Getting https://harrypotter.fandom.com/ru/wiki/%D0%A1%D0%BB%D1%83%D0%B6%D0%B5%D0%B1%D0%BD%D0%B0%D1%8F:%D0%92%D1%81%D0%B5_%D1%81%D1%82%D1%80%D0%B0%D0%BD%D0%B8%D1%86%D1%8B?from=1297..90 meaningful links
Getting https://harrypotter.fandom.com/ru/wiki/%D0%A1%D0%BB%D1%83%D0%B6%D0%B5%D0%B1%D0%BD%D0%B0%D1%8F:%D0%92%D1%81%D0%B5_%D1%81%D1%82%D1%80%D0%B0%D0%BD%D0%B8%D1%86%D1%8B?from=1551..131 meaningful links
Getting https://harrypotter.fandom.com/ru/wiki/%D0%A1%D0%BB%D1%83%D0%B6%D0%B5%D0%B1%D0%BD%D0%B0%D1%8F:%D0%92%D1%81%D0%B5_%D1%81%D1%82%D1%80%D0%B0%D0%BD%D0%B8%D1%86%D1%8B?from=1762+%D0%B3%D0%BE%D0%B4..185 meaningful links
Getting https://harrypotter.fandom.com/ru/wiki/%D0%A1%D0%BB%D1%83%D0%B6%D0%B5%D0%B1%D0%BD%D0%B0%D1%8F:%D0%92%D1%81%D0%B5_%D1%81%D1%82%D1%80%D0%B0%D0%BD%D0%B8%D1%86%D1%8B?from=1923..165 

In [41]:
import json
with open('pages.json','w',encoding='utf-8') as f:
    json.dump(pages,f,ensure_ascii=False)

In [51]:
import os
from tqdm import tqdm

for url,title in tqdm(pages.items()):
    title = title.replace('/','_').replace('?','.').replace(':','_')
    if os.path.exists(f'content/html/{title}.html'):
        #print(f"Skipping {title}")
        continue
    #print(f"Getting {title}..")
    html = requests.get("https://harrypotter.fandom.com"+url).text
    try:
        with open(f'content/html/{title}.html','w',encoding='utf-8') as f:
            f.write(html)
    except:
        print(f"WARNING: Cannot save {title}.html")


100%|██████████| 7288/7288 [00:15<00:00, 471.09it/s] 


In [45]:
def get_text(html):
    text = re.sub('<[^<]+?>', '', html)
    # Remove newlines and spacing between tags
    text = text.replace('\n', ' ').replace('  ', ' ') 
    # Split text into lines and remove leading and trailing whitespace  
    lines = (line.strip() for line in text.splitlines())
    text = '\n'.join(line for line in lines if line)
    return text


In [1]:
from bs4 import BeautifulSoup

def get_text(html):
    soup = BeautifulSoup(html, 'html.parser')

    title = soup.find('span', {'class' : 'mw-page-title-main'}).text
    body = soup.find('div',{ 'id' : 'mw-content-text'}).text
    
    # Get all text content
    #text = soup.get_text()
    return title,body

In [12]:
from tqdm import tqdm
import os, re
for fn in tqdm(os.listdir('content/html')):
    if re.match("\\d+ \\S+.html",fn):
        print(fn)

100%|██████████| 7323/7323 [00:00<00:00, 488382.89it/s]

1 августа.html
1 апреля.html
1 декабря.html
1 июля.html
1 июня.html
1 марта.html
1 мая.html
1 ноября.html
1 октября.html
1 сентября.html
1 февраля.html
1 января.html
10 августа.html
10 апреля.html
10 декабря.html
10 июля.html
10 июня.html
10 марта.html
10 мая.html
10 ноября.html
10 октября.html
10 сентября.html
10 февраля.html
10 января.html
1006 год.html
1014 год.html
1066 год.html
1097 год.html
11 августа.html
11 апреля.html
11 декабря.html
11 июля.html
11 июня.html
11 марта.html
11 мая.html
11 ноября.html
11 октября.html
11 сентября.html
11 февраля.html
11 января.html
1105 год.html
1107 год.html
1112 год.html
1163 год.html
1174 год.html
1189 год.html
1199 год.html
12 августа.html
12 апреля.html
12 декабря.html
12 июля.html
12 июня.html
12 марта.html
12 мая.html
12 ноября.html
12 октября.html
12 сентября.html
12 февраля.html
12 января.html
1202 год.html
1203 год.html
1207 год.html
1214 год.html
1227 год.html
1269 год.html
1285 год.html
1286 год.html
1289 год.html
1291 год.html
1292 г

In [30]:
class_info = set()

for fn in tqdm(os.listdir('content/html')):
    if re.match("\\d+ \\S+.html",fn):
        continue
    html = open(f'content/html/{fn}','r',encoding='utf-8').read()
    if len(html)<10:
        continue
    try:
        title, body = get_text(html)
    except:
        print(f"Error processing {fn}")
    body = re.sub(r'[\r\n]{4,}',r'\r\n',body)
    if 'п•о•р' in body:
        x = body[body.find('п•о•р'):]
        body = body[:body.find('п•о•р')]
        for t in x.split('п•о•р'):
            class_info.add(t)
    with open(f'content/text/{fn[:-5]}.txt','w',encoding='utf-8') as f:
        f.write(f"## {title}\n")
        f.write(body)        

  0%|          | 0/7323 [00:00<?, ?it/s]

100%|██████████| 7323/7323 [16:32<00:00,  7.38it/s] 


In [31]:
class_info

{'',
 '«Гарри Поттер» (серия)\n\n\nКанонические\n\nКниги серии\n\nФилософский камень · Тайная комната · Узник Азкабана · Кубок Огня · Орден Феникса · Принц-полукровка · Дары Смерти · Проклятое дитя\n\n\nДругие книги\n\nФантастические звери и места их обитания ·  Квиддич с древности до наших дней ·  Сказки барда Бидля · Гарри Поттер: Предыстория · Pottermore Presents · Хогвартс: неполный и ненадёжный путеводитель · Короткие рассказы из Хогвартса о власти, политике и назойливых полтергейстах · Короткие рассказы из Хогвартса о героизме, лишениях и опасных хобби · Фантастические твари и где они обитают: Оригинальный сценарий · Фантастические твари: Преступления Грин-де-Вальда: Оригинальный сценарий\n\n\nСайты\n\nWizarding World · Официальный сайт Джоан Роулинг\n\n\nWonderbook\n\nWonderbook: Книга заклинаний · Wonderbook: Книга зелий\n\n\nСпин-офф серия\n\nФантастические твари (серия фильмов)\n\n\nФильмы серии\n\nФилософский камень · Тайная комната · Узник Азкабана · Кубок Огня · Орден Фени

In [33]:
d = {}
for x in class_info:
    name = x.split('\r')[0].split('\n')[0]
    if name not in d:
        d[name] = x
    else:
        if len(x)<len(d[name]):
            d[name] = x
for k,v in d.items():
    with open(f'content/text/class_{k}.txt','w',encoding='utf-8') as f:
        f.write(v)